In [1]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
)
from azure.ai.ml.entities import (
    AmlCompute, Model, Environment, CodeConfiguration, 
    ManagedOnlineEndpoint, ManagedOnlineDeployment, OnlineRequestSettings, ProbeSettings
)
from azure.ai.ml.constants import AssetTypes

import time, sys, json, os
from IPython.display import display, JSON

In [2]:
credential = DefaultAzureCredential()

workspace_ml_client = MLClient.from_config(
    credential
)

Found the config file in: /config.json


In [3]:
# register the foundation model from local as Model asset in ml workspace
model = Model(
    path="models",
    type=AssetTypes.CUSTOM_MODEL,
    name="tinyllama-gguf",
    description="tinyllamagguf - gguf format model",
)

workspace_ml_client.create_or_update(model)

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpuds11001/code/Users/pupanda/gpt-use-case/foundation-models/quantized-model-inference/inference-tinyllama1.1b-gguf/models' 'https://mlws012181044126.blob.core.windows.net/azureml-blobstore-b81b1c5e-0151-42cf-9c96-ad079150a5ee/LocalUpload/f41ffe73a047bcb05789509cc20371db/models' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'tinyllama-gguf', 'description': 'tinyllamagguf - gguf format model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourceGroups/rg-mlws/providers/Microsoft.MachineLearningServices/workspaces/mlws01/models/tinyllama-gguf/versions/3', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpuds11001/code/Users/pupanda/gpt-use-case/foundation-models/quantized-model-inference/inference-tinyllama1.1b-gguf', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fc12d327b80>, 'serialize': <msrest.serialization.Serializer object at 0x7fc12d38b370>, 'version': '3', 'latest_version': None, 'path': 'azureml://subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourceGroups/rg-mlws/workspaces/mlws01/datastores/workspacebl

In [4]:
from azure.ai.ml.entities import Environment, BuildContext

# Create a new environment based on the curated one
custom_env = Environment(
    name="tinyllama-gguf-env",
    description="Custom environment with additional dependencies",
    build=BuildContext(path="../env")
)

# Register the custom environment
workspace_ml_client.environments.create_or_update(custom_env)

Uploading env (0.0 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 847/847 [00:00<00:00, 102176.52it/s]


Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'tinyllama-gguf-env', 'description': 'Custom environment with additional dependencies', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourceGroups/rg-mlws/providers/Microsoft.MachineLearningServices/workspaces/mlws01/environments/tinyllama-gguf-env/versions/14', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cpuds11001/code/Users/pupanda/gpt-use-case/foundation-models/quantized-model-inference/inference-tinyllama1.1b-gguf', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fc12d3bc460>, 'serialize': <msrest.serialization.Serializer object at 0x7fc12d38b550>, 'version': '14', 'conda_file': None, 'build': <azure.ai.

In [3]:
# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
online_endpoint_name = "tinyllama-gguf-ep"

In [4]:
# managed endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for tinyllama gguf",
    auth_mode="key",
)

In [7]:
# managed endpoint create async call
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [5]:
deployment_name = "deploy01"
model = "tinyllama-gguf@latest"
env = "tinyllama-gguf-env@latest"

In [13]:
# managed endpoint deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../onlinescoring",
        scoring_script="score.py",
    ),
    instance_type="Standard_F16s_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=120000,
    ),
    liveness_probe=ProbeSettings(
        initial_delay=600
    ),
)

In [14]:
# managed endpoint deplyment create async call
workspace_ml_client.online_deployments.begin_create_or_update(deployment=demo_deployment).wait()

Check: endpoint tinyllama-gguf-ep exists


Uploading onlinescoring (0.02 MBs): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19826/19826 [00:00<00:00, 182064.79it/s]


..................................................

In [11]:
# update traffic to the deployment for 100%
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://tinyllama-gguf-ep.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://tinyllama-gguf-ep.eastus2.inference.ml.azure.com/swagger.json', 'name': 'tinyllama-gguf-ep', 'description': 'Online endpoint for tinyllama gguf', 'tags': {}, 'properties': {'createdBy': 'Purna Chandra Panda', 'createdAt': '2024-12-23T03:32:43.436805+0000', 'lastModifiedAt': '2024-12-23T03:32:43.436805+0000', 'azureml.onlineendpointid': '/subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/resourcegroups/rg-mlws/providers/microsoft.machinelearningservices/workspaces/mlws01/onlineendpoints/tinyllama-gguf-ep', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6977e295-0d7c-4557-8e0b-26e2f6532103/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:b81b1c5e-0151-42cf-9c96-ad079150a5ee:3023fd76-3efc-48f0-995b-af942cb6d3fa?api-version=2022-02-

### Test endpoint with sample data

In [17]:
# score the request file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request1.json",
)

print(response)

{"output": "1. The Eiffel Tower\n2. Notre-Dame Cathedral\n3. Palace of Versailles\n4. Le Louvre Museum\n5. Champs-\u00c9lys\u00e9es\n6. Arc de Triomphe\n7. Sacr\u00e9-C\u0153ur Basilica\n8. Luxembourg Gardens\n9. Jardin des Plantes\n10. The Marais Poissoneries\n\nI hope this helps you plan your trip to Paris! Let me know if there's anything else I can help with."}


In [16]:
# score the request file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request3.json",
)

print(response)

{"output": "The capital of India is New Delhi."}
